In [1]:
import torch
import sys
import pickle
sys.path.append('../../data_processing')
sys.path.append('../../model')
sys.path.append('../../../data')
import CNN_LSTM
import TF_data_loader
from torch.utils import data as D
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
torch.set_printoptions(threshold=torch.inf)
np.set_printoptions(threshold=np.inf)

In [3]:
test_set = TF_data_loader.TF_data(data_path = '../../../data/Chipseq_data/seq_normalized_breathing_feat.pkl', partition = 'test' )
test_loader = D.DataLoader(test_set, batch_size = 128, num_workers = 0)

In [4]:
null_test_set = TF_data_loader.TF_data(data_path = '../../../data/Chipseq_data/seq_gaussiannoise_breathing_feat.pkl', partition = 'test' )
null_test_loader = D.DataLoader(null_test_set, batch_size = 128, num_workers = 0)

In [5]:
model_path = '../../../trained_model/DSTF_CNN/TF_biofeat_hybrid_n'
device = torch.device('cuda')
check_point = torch.load(model_path, map_location=device)
hybrid_model = CNN_LSTM.CNN_LSTM(num_target=130, input_channels = 7)
hybrid_model.load_state_dict(check_point['state_dict'])
hybrid_model = hybrid_model.cuda(0)

In [6]:
null_model_path = '../../../trained_model/DSTF_CNN/TF_biofeat_hybrid_g'
null_device = torch.device('cuda')
null_check_point = torch.load(null_model_path, map_location=null_device)
null_hybrid_model = CNN_LSTM.CNN_LSTM(num_target=130, input_channels = 7)
null_hybrid_model.load_state_dict(null_check_point['state_dict'])
null_hybrid_model = null_hybrid_model.cuda(0)

In [7]:
feat_map = torch.load('../../../data/Chipseq_data/FeatMap.pt')
feat_map = {v: k for k, v in feat_map.items()}

In [8]:
def onehot2seq(matrix):
    seq = ''
    for i in range(matrix.shape[1]):
        if matrix[0][i] == 1:
            nt = "A"
        elif matrix[1][i] == 1:
            nt = "G"
        elif matrix[2][i] == 1:
            nt = "C"
        elif matrix[3][i] == 1:
            nt = "T"
        seq += nt
    return(seq)

In [9]:
# cell_line_index = [i for i in range(130)]

# sorted_sequence_score_compendium = [[] for i in cell_line_index]
# sorted_sequence_score_topten = [[] for i in cell_line_index]
# sequence_score = [[] for i in cell_line_index]

# pbar2 = tqdm(total = len(test_loader))

# for seq, bio_feat, label in test_loader:
#     pbar2.update()
#     null_biofeat = torch.zeros_like(bio_feat)
#     input_feat = torch.cat([seq.cuda(0), bio_feat.cuda(0)], dim = 1).requires_grad_()
#     null_input_feat = torch.cat([seq.cuda(0), null_biofeat.cuda(0)], dim = 1).requires_grad_()
#     pred_tf = hybrid_model(input_feat)
#     null_pred_tf = hybrid_model(null_input_feat)
#     diff_tf = pred_tf - null_pred_tf
#     for i in range(label.size(0)):
#         for index in cell_line_index:
#             if label[i, index].item() == 1:
#                 sequence = onehot2seq(seq[i])
#                 diff = diff_tf[i, index].item()
#                 pred_tf_c = pred_tf[i, index].item()
#                 null_pred_tf_c = null_pred_tf[i, index].item()
#                 feature_vector = bio_feat[i]
#                 sequence_score[index].append([feat_map[index], diff, pred_tf_c, null_pred_tf_c, sequence, feature_vector])

# pbar2.close()

# for n, index in enumerate(sequence_score):
#     sorted_index = sorted(index, key=lambda x: x[1], reverse=True)
#     sorted_sequence_score_compendium[n] = sorted_index
#     sorted_sequence_score_topten[n] = sorted_index[:10]

In [10]:
# sorted_sequence_score_compendium = []
sorted_sequence_score_topten = []

In [11]:
cell_line_index = [i for i in range(130)]
pbar = tqdm(total = len(cell_line_index))

for index in cell_line_index:
    pbar.update()
    sequence_score = []
    for (seq, bio_feat, label), (null_seq, null_bio_feat, null_label) in zip(test_loader, null_test_loader):
        # null_biofeat = torch.zeros_like(bio_feat)
        input_feat = torch.cat([seq.cuda(0), bio_feat.cuda(0)], dim = 1).requires_grad_()
        null_input_feat = torch.cat([null_seq.cuda(0), null_bio_feat.cuda(0)], dim = 1).requires_grad_()
        pred_tf = hybrid_model(input_feat)
        null_pred_tf = null_hybrid_model(null_input_feat)
        diff_tf = pred_tf - null_pred_tf
        for i in range(label.size(0)):
            if label[i, index].item() == 1:
                sequence = onehot2seq(seq[i])
                diff = diff_tf[i, index].item()
                pred_tf_c = pred_tf[i, index].item()
                null_pred_tf_c = null_pred_tf[i, index].item()
                feature_vector = bio_feat[i]
                sequence_score.append([feat_map[index], diff, pred_tf_c, null_pred_tf_c, sequence, feature_vector])
    sorted_sequence_score = sorted(sequence_score, key=lambda x: x[1], reverse=True)
    # sorted_sequence_score_compendium.append(sorted_sequence_score)
    sorted_sequence_score_topten.append(sorted_sequence_score[:10])

  0%|          | 0/130 [00:00<?, ?it/s]

In [12]:
# with open('normalized_topten_sequences.pickle', 'wb') as f:
#     pickle.dump(sorted_sequence_score_compendium, f)
#     pickle.dump(sorted_sequence_score_topten, f)

In [13]:
# torch.save(sorted_sequence_score_compendium, 'sorted_sequence_score_compendium.pt')
torch.save(sorted_sequence_score_topten, 'sorted_sequence_score_topten.pt')

In [17]:
YY1_sorted_sequence_score_topten = []
other_sorted_sequence_score_topten = []

In [18]:
for cell_line in sorted_sequence_score_topten:
    if 'YY1' in cell_line[0][0]:
        YY1_sorted_sequence_score_topten.append(cell_line)
    else:
        other_sorted_sequence_score_topten.append(cell_line)

In [33]:
torch.save(YY1_sorted_sequence_score_topten, 'YY1_sorted_sequence_score_topten.pt')
torch.save(other_sorted_sequence_score_topten, 'other_sorted_sequence_score_topten.pt')